# Membership Inference Attack On Point Basis (Privacy Risk Score)

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/privML/privacy-evaluator/blob/main/notebooks/membership_inference_on_point_basis.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/team1sprint5/notebooks/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/privML/privacy-evaluator/blob/main/notebooks/membership_inference_on_point_basis.ipynb"><img src="https://raw.githubusercontent.com/privML/privacy-evaluator/main/notebooks/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


## Overview

In this notebook, we want to show you how to use the `privacy-evaluator` tool to perform the Membership Inference Attacks On Point Basis also known as calculating the privacy risk score on both, a provided PyTorch and a provided Tensorflow model.


## Setup

First, you should set the notebook's runtime to use a GPU (e.g. if Colab is used go to ***Runtime > Change runtime type > Hardware accelerator***). Now we can install the `privacy-evaluator` package and import all needed modules.

In [ ]:
!pip3 install git+https://github.com/privML/privacy-evaluator@team1sprint5

In [ ]:
import tensorflow as tf
import torch
import numpy as np

import tensorflow.python.ops.numpy_ops.np_config as np_config
np_config.enable_numpy_behavior()

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti
import privacy_evaluator.models.tf.dcti.dcti as tf_dcti 

from privacy_evaluator.datasets.tf.cifar10 import TFCIFAR10
from privacy_evaluator.datasets.torch.cifar10 import TorchCIFAR10

from privacy_evaluator.classifiers.classifier import Classifier

from privacy_evaluator.attacks.membership_inference.on_point_basis import MembershipInferenceAttackOnPointBasis
from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis

from privacy_evaluator.output.user_output_privacy_score import UserOutputPrivacyScore

from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis

from privacy_evaluator.attacks.membership_inference.data_structures.attack_input_data import AttackInputData
from privacy_evaluator.attacks.membership_inference.data_structures.slicing import Slicing


## Conduct Membership Inference Attack On Point Basis

Now we can start with conducting the Membership Inference Attacks On Point Basis. Therefore, we prepared two instances of the attack: one attacking a PyTorch model and attacking a TensorFlow model. For both attacks, we implemented a simple neural network trained on the CIFAR-10 dataset. For details about the provided network have a look at the following paper: https://www.scitepress.org/Papers/2018/67520/67520.pdf).

### PyTorch

We start the evaluation of the PyTorch version of the model.

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `torch.nn.CrossEntropyLoss`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize PyTorch model as a Classifier
target_model = Classifier(
    torch_dcti.load_dcti(), # PyTorch DCTI 
    loss=torch.nn.CrossEntropyLoss(reduction="none"), # Loss function of the PyTorch model
    nb_classes=TorchCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TorchCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Load CIFAR10 Dataset

Before we can start to conduct the membership inference attacks, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the PyTorch model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TorchCIFAR10.numpy()

#### Perform Membership Attack on Point Basis

Next, we want to perform a Membership Inference Attack on Point Basis its output is also known as Privacy Risk Score. In this case, we do not need to fit the attack because this approach depends only on the attacked the loss calculation of the data points and the target model.

To do so, we input the target model and the data points which should be evaluated to the respective function. The given data points are separated into a train and tests set. The train set contains of the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The test set contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model. As a result, we get privacy risk scores for each data point, separated into train and test scores. The resulting values indicate the probability of a data point being a member or not.

In [ ]:
attack = MembershipInferenceAttackOnPointBasis(target_model, x_train[:400], y_train[:400], x_test[:400], y_test[:400])

# Compute membership privacy risk score for the PyTorch model


(train_privacy_risk_score, test_privacy_risk_score) = attack.attack()

#### Get human-readable privacy risk score statistics

We can create more human-readable privacy risk score statistics. Therefore we generate an output by providing the privacy risk scores and the true labels of the data points for which we computed the privacy risk scores. This output can then be visualized in two separate ways.

In [ ]:
# Create user output and plot histogram for train dataset
output = UserOutputPrivacyScore(
    np.argmax(y_train[:400], axis=1),
    train_privacy_risk_score, 
)

The first way to visualise the privacy risk scores is as a histogram. The histogram shows the distribution of the k-top data points with the highest privacy risk scores per class.

In [ ]:
# Plot absolut values 
labels, count = output.histogram_top_k(range(10), 50)

As a second option, you can visualise the privacy risk scores again as a histogram of the distribution of the k-top data points with the highest privacy risk scores per class, but this time the values are relative to the size of respective classe

In [ ]:
# Plot relative values 
labels, count = output.histogram_top_k_relative(range(10), 50)

#### Explanation of the outcome:

##### Vulnerability of individual data points:
The training data points that exhibit an increased membership privacy risk might differ from their classes mean samples (outliers) [7]. You could check them again, see if they have the correct label, or if they exhibit any non-standard properties for the class. If so, correct them. It was also shown that points with an high influence on the decision boundary are more vulnerable to membership inference attacks [5]. Therefore, these points might be important. If you want to protect them, you might add several similar training samples as they are to the class. 


(For References, please see last box)

### TensorFlow

Now we do the same with the TensorFlow model.

#### Prepare target model

Now, we need to initialize our pre-trained Lightweight Deep Convolutional Neural Network (short DCTI) as a generic `Classifier`. Therefore we need to specify the loss function used to train the model (in our case the `tf.keras.losses.CategoricalCrossentropy`), the number of classes and the input shape of our CIFAR-10 dataset.

In [ ]:
# Initalize TensorFlow target model
target_model = Classifier(
    tf_dcti.load_dcti(), # TensorFlow DCTI
    loss=tf.keras.losses.CategoricalCrossentropy(), # Loss function of the TensorFlow target model
    nb_classes=TFCIFAR10.N_CLASSES, # Number of classes of the CIFAR10 dataset
    input_shape=TFCIFAR10.INPUT_SHAPE # Input shape of the CIFAR10 dataset
)

#### Load CIFAR10 Dataset

Again, before we can start to conduct the membership inference attacks, we need to load the dataset. The CIFAR10 dataset needs to be preprocessed in a specific manner to work for the TensorFlow model.

In [ ]:
# Load CIFAR10 dataset as numpy array
x_train, y_train, x_test, y_test = TFCIFAR10.numpy()

#### Perform Membership Attack on Point Basis

Next, we want to perform a Membership Inference Attack on Point Basis its output also known as Privacy Risk Score. In this case, we do not need to fit the attack because this approach depends only on the attacked the loss calculation of the data points and the target model.

To do so, we input the target model and the data points which should be evaluated to the respective function. The given data points are separated into a train and tests set. The train set contains of the data (`x_train`) and its corresponding labels (`y_train`) which were used to train the target model. The test set contains the data (`x_test`) and its corresponding labels (`y_test`) which were not part of the training process of the target model. As a result, we get privacy risk scores for each data point, separated into train and test scores. The resulting values indicate the probability of a data point being a member or not.

In [ ]:
attack = MembershipInferenceAttackOnPointBasis(target_model, x_train[:100], y_train[:100], x_test[:100], y_test[:100])

# Compute membership privacy risk score for the PyTorch model


(train_privacy_risk_score, test_privacy_risk_score) = attack.attack()

#### Get human-readable privacy risk score statistics

We can create more human-readable privacy risk score statistics. Therefore we generate an output by providing the privacy risk scores and the true labels of the data points for which we computed the privacy risk scores. This output can then be visualized in two separate ways.

In [ ]:
# Create user output and plot histogram for train dataset
output = UserOutputPrivacyScore(
    np.argmax(y_train[:100], axis=1), 
    train_privacy_risk_score, 
)

The first way to visualise the privacy risk scores is as a histogram. The histogram shows the distribution of the k-top data points with the highest privacy risk scores per class.

In [ ]:
# Plot absolut values 
labels, count = output.histogram_top_k(range(10), 50)

As a second option, you can visualise the privacy risk scores again as a histogram of the distribution of the k-top data points with the highest privacy risk scores per class, but this time the values are relative to the size of respective classe

In [ ]:
# Plot relative values 
labels, count = output.histogram_top_k_relative(range(10), 50)

#### Explanation of the outcome:

##### Vulnerability of individual data points:
The training data points that exhibit an increased membership privacy risk might differ from their classes mean samples (outliers) [7]. You could check them again, see if they have the correct label, or if they exhibit any non-standard properties for the class. If so, correct them. It was also shown that points with an high influence on the decision boundary are more vulnerable to membership inference attacks [5]. Therefore, these points might be important. If you want to protect them, you might add several similar training samples as they are to the class. 


(For References, please see last box)

[1]S. Yeom, I. Giacomelli, M. Fredrikson, and S. Jha. \Privacy Risk in Machine Learning: Analyzing the Connection to Overfitting". In: 2018 IEEE 31st Computer Security Foundations Symposium (CSF). July 2018, pp. 268{282. doi:10.1109/CSF.2018.00027.

[2] Reza Shokri, Marco Stronati, Congzheng Song, and Vitaly Shmatikov. 2017. Mem-bership Inference Attacks Against Machine Learning Models. In2017 IEEE Sym-posium on Security and Privacy (SP). 3–18.

[3] Milad Nasr, Reza Shokri, and Amir Houmansadr. 2018. Machine Learning withMembership Privacy Using Adversarial Regularization. InProceedings of the 2018ACM SIGSAC Conference on Computer and Communications Security(Toronto,Canada)(CCS ’18). Association for Computing Machinery, New York, NY, USA,634–64

[4] Cynthia Dwork. 2006.  Differential Privacy. InAutomata, Languages and Pro-gramming, Michele Bugliesi, Bart Preneel, Vladimiro Sassone, and Ingo Wegener(Eds.). Springer Berlin Heidelberg, Berlin, Heidelberg

[5] Stacey Truex, Ling Liu, Mehmet Emre Gursoy, Lei Yu, and Wenqi Wei. 2019.Demystifying Membership Inference Attacks in Machine Learning as a Service.IEEE Transactions on Services Computing(2019)

[6] Suriyakumar, Vinith M., Nicolas Papernot, Anna Goldenberg, and Marzyeh Ghassemi. "Chasing Your Long Tails: Differentially Private Prediction in Health Care Settings." In Proceedings of the 2021 ACM Conference on Fairness, Accountability, and Transparency, pp. 723-734. 2021.

[7] Yunhui Long, Vincent Bindschaedler, Lei Wang, Diyue Bu, Xiaofeng Wang, HaixuTang, Carl A. Gunter, and Kai Chen. 2018.   Understanding Membership In-ferences on Well-Generalized Learning Models.CoRRabs/1802.04889 (2018).arXiv:1802.04889  http://arxiv.org/abs/1802.0
